## Downloading data from Bloomberg API 

In [2]:
from xbbg import blp
import pandas as pd
from datetime import datetime

Skipped: could not import 'blpapi': No module named 'blpapi'

In [ ]:
def get_option_data(tickers, output_file="options_data.csv"):
    """
    Fetches option chain data including tickers, expiry, strike, type, market data, and Greeks for given tickers.

    Parameters:
    - tickers: list of str, Bloomberg symbols (e.g., ["SPY US Equity", "NVDA US Equity", "VIX Index"])
    - output_file: str, Name of the output CSV file.

    Returns:
    - DataFrame containing all option data.
    """

    start_date = datetime.today()
    cutoff_date = start_date + timedelta(days=90)


    all_options = []

    for ticker in tickers:
        # get all option contracts
        option_chain = blp.bds(f"{ticker} Equity", "OPT_CHAIN")

        # continue to next ticker if no option is found
        if option_chain.empty:
            print(f"No options found for {ticker}")
            continue

        # extract option tickers
        option_tickers = option_chain["security_description"].tolist()

        # fetch Market Data
        fields = [
            "OPT_EXPIRE_DT","OPT_STRIKE_PX", "OPT_PUT_CALL",
            "PX_BID", "PX_ASK", "PX_MID", "PX_LAST",  # Market prices
            "IVOL_LAST", "DELTA_LAST", "GAMMA_LAST", "VEGA_LAST", "THETA_LAST", "RHO_LAST"  # Greeks
        ]

        # might have to iterate all the option_ticker
        option_data = blp.bdp(option_tickers, fields)

        if option_data.empty:
            print(f"No market data available for options of {ticker}")
            continue

        # format DataFrame
        option_data.reset_index(inplace=True)
        option_data.rename(columns={"index": "Option_Ticker"}, inplace=True)

        # convert Expiry Date to readable format
        option_data["OPT_EXPIRE_DT"] = pd.to_datetime(option_data["OPT_EXPIRE_DT"])

        # filter out the options expiring in the next three months
        option_data = option_data[
            (option_data["OPT_EXPIRE_DT"] >= start_date) & 
            (option_data["OPT_EXPIRE_DT"] <= cutoff_date)
        ]

        # store data
        all_options.append(option_data)

    # combine all options into one DataFrame
    if all_options:
        final_df = pd.concat(all_options, ignore_index=True)

        # save to CSV
        final_df.to_csv(output_file, index=False)
        print(f"Option data saved to {output_file}")

        return final_df
    else:
        print("No option data collected.")
        return pd.DataFrame()

In [4]:
# Example Usage
tickers = ["SPY US Equity", "NVDA US Equity", "VIX Index"]
options_df = get_option_data(tickers, "options_data.csv")

NameError: name 'blp' is not defined